# Twibbonizer

In [1]:
from PIL import Image
import os


In [2]:
# Paths
input_folders = ["data/katalog_1", "data/katalog_2"]
output_base = "output"
twibbon_path = "data/twibbon_1.png"
watermark_path = "data/watermark_1.png"

# Ensure output directory exists
os.makedirs(output_base, exist_ok=True)

In [3]:
# Load twibbon and watermark
twibbon = Image.open(twibbon_path).convert("RGBA")
watermark = Image.open(watermark_path).convert("RGBA")


In [4]:
# Define the target size
target_size = (1080, 1080)
watermark_opacity = 0.7

In [5]:
# Process each katalog folder
for katalog in input_folders:
    output_path = os.path.join(output_base, os.path.basename(katalog))  # Fix output path
    os.makedirs(output_path, exist_ok=True)

    # Process all images in katalog folder
    for filename in os.listdir(katalog):
        if filename.lower().endswith((".jpg", ".jpeg", ".png", ".webp")):
            img_path = os.path.join(katalog, filename)
            img = Image.open(img_path).convert("RGBA")

            # Resize images to 1080x1080
            img_resized = img.resize(target_size, Image.LANCZOS)
            twibbon_resized = twibbon.resize(target_size, Image.LANCZOS)

            # Resize watermark to 50% of image size
            watermark_size = (target_size[0] // 2, target_size[1] // 2)
            watermark_resized = watermark.resize(watermark_size, Image.LANCZOS)

            # Adjust watermark opacity
            watermark_transparent = Image.new("RGBA", watermark_resized.size)
            for x in range(watermark_resized.width):
                for y in range(watermark_resized.height):
                    r, g, b, a = watermark_resized.getpixel((x, y))
                    watermark_transparent.putpixel((x, y), (r, g, b, int(a * watermark_opacity)))

            # Create blank canvas with full-size transparent background
            watermark_layer = Image.new("RGBA", target_size, (0, 0, 0, 0))
            
            # Position watermark at center
            position = ((target_size[0] - watermark_size[0]) // 2, (target_size[1] - watermark_size[1]) // 2)
            watermark_layer.paste(watermark_transparent, position, mask=watermark_transparent)

            # Overlay twibbon on product image
            combined = Image.alpha_composite(img_resized, twibbon_resized)

            # Overlay the watermark layer on top
            final_image = Image.alpha_composite(combined, watermark_layer)

            # Determine output format
            output_format = "PNG" if filename.lower().endswith(".png") else "JPEG"
            output_ext = ".png" if output_format == "PNG" else ".jpg"

            # Save final image in /output/katalog_x
            output_file = os.path.join(output_path, os.path.splitext(filename)[0] + output_ext)
            final_image.convert("RGB").save(output_file, format=output_format, quality=95)

            print(f"Processed: {filename} -> {output_file}")

print("✅ All images processed and saved in /output/katalog_x folders!")

Processed: produk_1.jpg -> output\katalog_1\produk_1.jpg
Processed: produk_2.jpeg -> output\katalog_1\produk_2.jpg
Processed: produk_3.jpg -> output\katalog_1\produk_3.jpg
Processed: produk_4.jpg -> output\katalog_1\produk_4.jpg
Processed: produk_5.jpg -> output\katalog_1\produk_5.jpg
Processed: bunga_1.jpg -> output\katalog_2\bunga_1.jpg
Processed: bunga_2.webp -> output\katalog_2\bunga_2.jpg
Processed: bunga_3.jpg -> output\katalog_2\bunga_3.jpg
Processed: bunga_4.jpg -> output\katalog_2\bunga_4.jpg
Processed: bunga_5.jpg -> output\katalog_2\bunga_5.jpg
✅ All images processed and saved in /output/katalog_x folders!
